In [ ]:
# ==================== IMPORTS ====================
from flask import Flask, request, jsonify, render_template
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
from werkzeug.utils import secure_filename
from PIL import Image

# ==================== PATHS ====================
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
MODEL_DIR = os.path.join(BASE_DIR, 'models')
UPLOAD_FOLDER = os.path.join(BASE_DIR, 'uploads')
TEMPLATE_FOLDER = os.path.join(BASE_DIR, 'templates')

# ==================== APP SETUP ====================
app = Flask(__name__, template_folder=TEMPLATE_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# ==================== MODEL LOADING ====================
image_model = tf.keras.models.load_model(os.path.join(MODEL_DIR, 'asd_image_model.keras'))
audio_model = tf.keras.models.load_model(os.path.join(MODEL_DIR, 'asd_audio_model.keras'))

# ==================== AUDIO LOADING FUNCTION ====================
def load_audio(audio_path, max_length=100):
    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc = mfcc.T
    if mfcc.shape[0] > max_length:
        mfcc = mfcc[:max_length, :]
    else:
        pad_width = max_length - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    return mfcc

# ==================== ROUTES ====================
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Process image
        image_file = request.files['image']
        image_path = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(image_file.filename))
        image_file.save(image_path)

        img = load_img(image_path, target_size=(128, 128))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        image_pred = image_model.predict(img_array)[0][0]
        image_label = "Autism Detected" if image_pred > 0.5 else "No Autism"

        # Process audio
        audio_file = request.files['audio']
        audio_path = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(audio_file.filename))
        audio_file.save(audio_path)

        audio_data = np.expand_dims(load_audio(audio_path), axis=0)
        audio_pred = audio_model.predict(audio_data)[0][0]
        audio_label = "Autism Detected" if audio_pred > 0.5 else "No Autism"

        final_result = f"Image: {image_label} ({image_pred * 100:.2f}%) | Audio: {audio_label} ({audio_pred * 100:.2f}%)"

        return jsonify({'prediction': final_result})

    except Exception as e:
        print(e)
        return jsonify({'error': 'Prediction failed!'})

# ==================== MAIN ====================
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 * ngrok tunnel link: NgrokTunnel: "https://0d0a-34-32-168-110.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0d0a-34-32-168-110.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 20:47:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 20:47:03] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step


...

INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 20:54:49] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 20:55:16] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 20:55:23] "POST /predict HTTP/1.1" 200 -


In [ ]:
import os

# Check if the file really exists
os.path.exists('/content/drive/MyDrive/AutismDetectionProject/webapp/templates/index.html')

True

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok